In [ ]:
import math
from tabulate import tabulate
import copy
import numpy as np
import pandas as pd
import random
import string
import statistics


# Función auxiliar para inyectar métodos a una clase
# Nos permitirá añadir métodos a la clase más adelante en el notebook
def add_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator



class FinancialAsset:
    """
    Representa un activo financiero.
    """
    def __init__(self, name, price, roi, variance=0):
        self.name = name
        self.price = price       # Coste por unidad (c)
        self.roi = roi           # Retorno esperado (r) como ratio (ej. 1.05)
        self.variance = variance # Riesgo intrínseco (sigma^2)

    def __str__(self):
        return self.name

class DecisionVariable:
    """
    Representa una variable para nuestro QUBO. Pueden ser un mapeo directo de FinancialAsset
    o otro tipo de variables (holgura, descomposición binaria...)
    """
    def __init__(self, parent_name, quantity, cost, value, is_slack=False):
        self.parent_name = parent_name # Nombre del activo "padre" de este
        self.quantity = quantity # Cantidad de activos (si representa varias unidades del mismo activo)
        self.cost = cost # Coste de esta variable (precio del activo por número de activos)
        self.value = value # Valor esperado total de esta variable (ROI * coste)
        self.is_slack = is_slack # Booleano para identificar variables de holgura
        self._full_name = self.parent_name + "_" + str(self.quantity)

    def name(self):
        return self._full_name

class PortfolioProblem:
    """
    Clase constructora del problema de optimización.
    """
    def __init__(self, max_budget):
        self.max_budget = max_budget
        self.assets = {} # indexados por nombre: name -> FinancialAsset
        self.covariances = {} # indexados por pares de nombres: (name, name) -> float
        self.variables = [] # Lista de DecisionVariable
        self.variables_index = None

    def copy(self):
        return copy.deepcopy(self)
    
    
    def add_asset(self, name, price, variance=0, roi_ratio=None, expected_value=None):
        """
        Registra un activo. Puede tomar un ROI o un valor esperado (y almacena el ROI calculado)
        """
        final_roi = roi_ratio
        if final_roi is None:
            if expected_value is None:
                final_roi = 0
            else:
                final_roi = expected_value / price

        self.assets[name] = FinancialAsset(name, price, final_roi, variance)

    def add_covariance(self, asset1_name, asset2_name, value):
        """Registra la covarianza entre dos activos."""
        key = tuple(sorted((asset1_name, asset2_name)))
        self.covariances[key] = value

    def get_asset_risk(self, name1, name2=None):
        """Devuelve la covarianza (o varianza si es el mismo activo)."""
        if name2 == None or name1 == name2:
            return self.assets[name1].variance
        key = tuple(sorted((name1, name2)))
        return self.covariances.get(key, 0.0)

    def get_variable_by_full_name(self, full_name):
        if self.variables_index == None or full_name not in self.variables_index:
            self.variables_index = {}
            for v in self.variables:
                self.variables_index[v._full_name]=v
        return self.variables_index[full_name]

    def clear_variables(self):
        """Limpia la lista de variables."""
        self.variables = []

    def create_variables_simple(self):
        """
        Crea las variables para un problema mochila básico: Una variable por activo.
        """
        for asset in self.assets.values():
            if asset.price <= self.max_budget:
                val = asset.price * asset.roi
                self.variables.append(DecisionVariable(asset.name, 1, asset.price, val))
    
    def create_slack_variables(self):
        """
        Estrategia: Variables de holgura (Slack).
        """
        
        # Tenemos que sumar 1 ya que si el número máximo es justo una potencia de dos
        # no cubriríamos todo (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
        k = int(math.floor((math.log2(self.max_budget))+1))
        
        for i in range(k):
            val = 2**i                
            self.variables.append(DecisionVariable("SLACK", int(val), float(val), 0.0, is_slack=True))

    def print_problem_definition(self, display_covariances = False, display_variables=False):
        """
        Imprime los activos disponibles, covarianzas y estado de variables generadas.
        """
        print("\n=== Definición del Problema ===")
        
        print(f"Presupuesto Máximo (C): {self.max_budget}")

        # Tabla de Activos
        print("\n--- Activos Disponibles ---")
        table_data = []
        for a in self.assets.values():
            table_data.append([a.name, a.price, f"{a.roi:.4f}", a.variance])
        
        print(tabulate(table_data, headers=["Activo", "Coste Unit.", "ROI (Ratio)", "Riesgo intrínseco (Varianza)"]))
            
        # Tabla de Covarianzas
        if display_covariances and self.covariances:
            print("\n--- Riesgos Conjuntos (Covarianzas) ---")
            cov_data = [[k[0], k[1], v] for k, v in self.covariances.items()]
            print(tabulate(cov_data, headers=["Activo A", "Activo B", "Valor"]))

        # Resumen de Variables
        print("\n--- Estado del Modelo ---")
        print(f"Variables de decisión generadas: {len(self.variables)}")
        slack_count = sum(1 for v in self.variables if v.is_slack)
        if slack_count > 0:
            print(f"(Incluye {slack_count} variables de holgura)")
        if(display_variables):
            print("\n--- Variables ---")
            table_data = []
            for a in self.variables:
                table_data.append([a.name(), a.quantity, f"{a.cost:.4f}", f"{a.value:.4f}"])
            
            print(tabulate(table_data, headers=["Activo", "Unidades", "Coste", "Valor"]))  


    def get_solution_table(self, solution_vector):
        """
        Para un vector de solución dado, entrega un diccionario con todas las acciones, la
        cantidad a comprar, el coste total y el retorno esperado total:

        name -> {'quantity': 0, 'cost': 0.0, 'value': 0.0}
        """
        summary = {}
        for name, selected in solution_vector.items():
            if selected > 0:
                var = self.get_variable_by_full_name(name)
                
                if not var.is_slack:
                    if var.parent_name not in summary:
                        summary[var.parent_name] = {'quantity': 0, 'cost': 0.0, 'value': 0.0}
                    
                    summary[var.parent_name]['quantity'] += var.quantity
                    summary[var.parent_name]['cost'] += var.cost
                    summary[var.parent_name]['value'] += var.value
        return summary
        
    def print_solution(self, solution_vector):
        """
        Imprime la solución
        """
        print("\n=== Solución ===")
        if len(solution_vector) != len(self.variables):
            print("Error: Vector solución de tamaño incorrecto.")
            return

        summary = {}
        total_inv = 0
        total_ret = 0
        slack_inv = 0

        for name, selected in solution_vector.items():
            if selected > 0:
                var = self.get_variable_by_full_name(name)
                
                if var.is_slack:
                    slack_inv += var.cost
                else:
                    if var.parent_name not in summary:
                        summary[var.parent_name] = {'u': 0, 'c': 0.0, 'v': 0.0}
                    
                    summary[var.parent_name]['u'] += var.quantity
                    summary[var.parent_name]['c'] += var.cost
                    summary[var.parent_name]['v'] += var.value
                    
                    total_inv += var.cost
                    total_ret += var.value

        data = []
        for name, info in summary.items():
            pct = (info['c'] / self.max_budget) * 100
            data.append([name, info['u'], f"{info['c']:.2f}", f"{pct:.1f}%", f"{info['v']:.2f}"])
            
        if slack_inv > 0:
            data.append(["[HOLGURA]", "-", f"{slack_inv:.2f}", f"{(slack_inv/self.max_budget)*100:.1f}%", "0.0"])

        print(tabulate(data, headers=["Activo", "Unidades", "Inversión", "% Total", "Retorno"]))
        print(f"Total Invertido en Activos: {total_inv:.2f}")
        print(f"Total presupuesto: {self.max_budget:.2f}")
        print(f"Holgura (no invertido): {slack_inv:.2f}")
        print(f"Total invertido + holgura: {total_inv + slack_inv:.2f}", f"( {100*(total_inv + slack_inv)/self.max_budget:.2f} %)")
        print(f"Total retorno: {total_ret:.2f}")

        risk = 0
        active_assets = list(summary.keys())
        for n1 in active_assets:
            for n2 in active_assets:
                c1 = summary[n1]['c']
                c2 = summary[n2]['c']
                risk += c1 * c2 * self.get_asset_risk(n1, n2)
        print(f"Riesgo Total (Varianza): {risk:.4f}")

In [ ]:
## Código para generar problemas aleatorios y de ejemplo
from dimod import ExactSolver

def create_random_problem(max_investment, num_assets):    
    problema = PortfolioProblem(max_investment)
    
    for i in range (num_assets):
        name = ''.join(random.choice(string.ascii_uppercase) for _ in range(4))
        price = round(random.lognormvariate(3.5, 1.0), 2) # lognormvariate porque queremos precios positivos, con sigma alto para tener bastante dispersión
        roi_ratio = round(random.normalvariate(0.08, 0.20), 4) # Media 0.08 (8% histórico aprox) y desviación 0.20 (20% volatilidad estándar)
        roi_ratio = round(random.normalvariate(0.08, 0.20)**2, 4)
        variance = round(random.gammavariate(2, 0.04), 4) # Media de 0.08 con cola a la derecha. Valores muy concentrados pero con algún activo muy volatil
        problema.add_asset(name, price, variance, roi_ratio)
    
    for a1 in problema.assets.values():
        std_1 = math.sqrt(a1.variance) #volatilidad real, la raiz de la varianza
        for a2 in problema.assets.values():
            if a1.name >= a2.name: # Solo añadimos una diagonal de la matriz
                continue
            std_2 = math.sqrt(a1.variance) #volatilidad real, la raiz de la varianza
            rho = random.uniform(-0.2, 0.8) # La correlación es aleatoria
            cov = rho * std_1 * std_2 # Fórmula de la covarianza en función de la volatilidad y la correlación
            problema.add_covariance(a1.name, a2.name, cov)
    return problema
    
problema_random = create_random_problem(max_investment = 1000000, num_assets = 7)

problema_estatico = PortfolioProblem(200)

problema_estatico.add_asset("1", 100, 0.15, 0.1)
problema_estatico.add_asset("2", 200, 0.10, 0.05)
problema_estatico.add_asset("3", 50, 0.02, -0.05)
problema_estatico.add_asset("4", 50, 0.15, -0.25)
problema_estatico.add_asset("5", 75, 0.05, 0.15)
problema_estatico.add_asset("6", 75, 0.06, 0.13)

for a in problema_estatico.assets.values():
    for b in problema_estatico.assets.values():
        if (a.name == "1" and b.name =="5"):
            std_1 = math.sqrt(a.variance)
            std_2 = math.sqrt(b.variance)
            rho = 0.2
            cov = rho * std_1 * std_2
            problema_estatico.add_covariance(a.name, b.name, cov)
        elif (a.name == "5" and b.name =="1"):
            std_1 = math.sqrt(a.variance)
            std_2 = math.sqrt(b.variance)
            rho = 0.2
            cov = rho * std_1 * std_2
            problema_estatico.add_covariance(a.name, b.name, cov)
        elif (a.name == "5" and b.name =="6"):
            std_1 = math.sqrt(a.variance)
            std_2 = math.sqrt(b.variance)
            rho = -0.2
            cov = rho * std_1 * std_2
            problema_estatico.add_covariance(a.name, b.name, cov)
        elif (a.name == "6" and b.name =="5"):
            std_1 = math.sqrt(a.variance)
            std_2 = math.sqrt(b.variance)
            rho = -0.2
            cov = rho * std_1 * std_2
            problema_estatico.add_covariance(a.name, b.name, cov)
        else:
            problema_estatico.add_covariance(a.name, b.name, 0)

In [ ]:
# Resolvemos un QUBO básico para el problema anterior
# Usamos variables binarias (invertir o no) y NO incorporamos variables de holgura ni riesgo

def solve_qubo_basico_vars(problema):
    ## QUBO básico tipo mochila. Sin riesgo ni multi-inversiones
    qubo_basico = {}
    
    # Calculamos alpha buscando el valor máximo entre los objetos
    alpha = max([o.value for o in problema.variables]) * 1.1
    print ("Alpha: ", alpha)
    
    for o_i in problema.variables:
        # Término lineal
        qubo_basico[(o_i.name(), o_i.name())] = (
            - o_i.value
            + alpha * (o_i.cost**2)
            - 2 * alpha * problema.max_budget * o_i.cost
        )
    
        for o_j in problema.variables:
            if o_i.name() == o_j.name():
              continue
            # Término cuadrático
            qubo_basico[(o_i.name(), o_j.name())] = alpha * o_i.cost * o_j.cost
    exact_solver = ExactSolver()
    soluciones_qubo_basico = exact_solver.sample_qubo(qubo_basico)
    solucion_qubo_basico = soluciones_qubo_basico.first.sample
    problema.print_solution(solucion_qubo_basico)

In [ ]:
@add_method(PortfolioProblem)
def get_interaction_risk(self, var_i, var_j):
    """Calcula el riesgo cuadrático entre dos variables, teniendo en cuenta que pueden estar paquetizadas"""

    # Las variables slack no incrementan el riesgo, así que devolvemos 0
    if var_i.is_slack or var_j.is_slack:
        return 0.0

    # El riesgo aumenta con la cantidad de dinero invertido en cada una de las variables (ya que 
    # la covarianza está expresada para cantidad 1 de cada variable)
    cov = self.get_asset_risk(var_i.parent_name, var_j.parent_name)
    return var_i.cost * var_j.cost * cov

@add_method(PortfolioProblem)
def create_variables_binary_simple(self):
    """
    Crea las variables usando descomposición binaria. Se permite un número arbitrario de 
    unidades de cada activo.
    """
    for asset in self.assets.values():
        # Calculamos cuantas acciones puedo tener
        max_units = math.floor(self.max_budget / asset.price)

        # Si no me entra ni siquiera una acción, no añado variables
        if max_units < 1:
            continue
        # Tenemos que sumar 1 ya que si el número máximo es justo una potencia de dos
        # no cubriríamos todo (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
        k = int(math.floor((math.log2(max_units))+1))

        for i in range(k):
            count = 2**i
            cost = count * asset.price
            val = cost * asset.roi
            self.variables.append(DecisionVariable(asset.name, count, cost, val))



def solve_qubo_basico_risk(problema, alpha=None, beta=None):
    ## QUBO básico tipo mochila. Sin riesgo ni multi-inversiones
    qubo_basico = {}
    
    # Calculamos alpha buscando el valor máximo entre los objetos
    if alpha == None:
        alpha = max([o.value for o in problema.variables]) * 1.1
    print ("Alpha: ", alpha)

    avg_roi = statistics.mean([a.value 
                               for a in problema.variables
                               if not a.is_slack])
    avg_cov = statistics.mean([a.cost**2 * problema.get_asset_risk(a.parent_name) 
                               for a in problema.variables
                               if not a.is_slack])
    if beta == None:    
        beta = avg_roi/avg_cov
    print ("Beta: ", beta, avg_roi, avg_cov)
    
    for o_i in problema.variables:
        varianza = 0
        if not o_i.is_slack:
            varianza = problema.get_asset_risk(o_i.parent_name)
        # Término lineal
        qubo_basico[(o_i.name(), o_i.name())] = (
            - o_i.value
            + alpha * (o_i.cost**2)
            - 2 * alpha * problema.max_budget * o_i.cost
            + beta * varianza * o_i.cost**2 
        )
    
        for o_j in problema.variables:
            if o_i.name() == o_j.name():
              continue
            covarianza = 0
            if not o_i.is_slack and not o_j.is_slack:
                covarianza = problema.get_asset_risk(o_i.parent_name, o_j.parent_name)  
            # Término cuadrático
            qubo_basico[(o_i.name(), o_j.name())] = (  
                alpha * o_i.cost * o_j.cost
                + beta * o_i.cost * o_j.cost * covarianza
             )
    exact_solver = ExactSolver()
    soluciones_qubo_basico = exact_solver.sample_qubo(qubo_basico)
    solucion_qubo_basico = soluciones_qubo_basico.first.sample
    problema.print_solution(solucion_qubo_basico)
    return (alpha, beta, solucion_qubo_basico)


In [ ]:
@add_method(PortfolioProblem)
def create_variables_binary_packaged(self, min_package_pct=0.05):
    """
    Crea las variables usando descomposición binaria. Se permiten varias unidades de cada activo,
    pero se permite solo un número mínimo de unidades y múltiplos de ese número.
    Así, en caso de acciones muy baratas, reducimos el número de variables.
    El número mínimo de unidades se calcula en función del presupuesto total (tamaño de la cartera)
    """
    min_money_package = self.max_budget * min_package_pct
    
    for asset in self.assets.values():
        # Cuantas acciones son un paquete?
        if asset.price >= min_money_package:
            base_units = 1
        else:
            base_units = round(min_money_package / asset.price)

        # Cuantos paquetes nos entran en el presupuesto?
        max_packages =  math.floor(self.max_budget / (asset.price*base_units))

        # Si no entra ni siquiera un paquete, no añadimos esta variable
        if max_packages < 1:
            continue

        # Tenemos que sumar 1 ya que si el número máximo es justo una potencia de dos
        # no cubriríamos todo (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
        k = int(math.floor((math.log2(max_packages))+1))

        for i in range(k):
            package_multiplier = 2**i
            real_units = package_multiplier * base_units
            cost = real_units * asset.price
            val = cost * asset.roi
            self.variables.append(DecisionVariable(asset.name, real_units, cost, val))

@add_method(PortfolioProblem)
def create_slack_variables_packaged(self, min_package_pct=None ):
    """
    Estrategia: Variables de holgura (Slack), añadiendo el tamaño mínimo del paquete
    """
    if min_package_pct == None:
        self.create_slack_variables()
    else:
        min_money_package = self.max_budget * min_package_pct
        max_packages =  math.floor(self.max_budget / min_money_package)
        
        k = int(math.floor(math.log2(max_packages)+1))

        for i in range(k):
            quantity = 2**i
            val = quantity*min_money_package               
            self.variables.append(DecisionVariable("SLACK", int(quantity), float(val), 0.0, is_slack=True))   
        # Y una última variable con el relleno 
        holgura_final = self.max_budget - (max_packages * min_money_package)
        self.variables.append(DecisionVariable("SLACK", 0, float(holgura_final), 0.0, is_slack=True)) 


In [ ]:
@add_method(PortfolioProblem)
def create_variables_with_max_weight(self, max_weight_per_asset, min_package_pct=0.05):
    """
    Genera variables binarias sin cubrir hasta el máximo del presupuesto, cubriendo solo
    hasta un máximo relativo por activo.
    """
    min_money_package = self.max_budget * min_package_pct
    max_amt = max_weight_per_asset * self.max_budget
    if max_amt > self.max_budget:
        max_amt = self.max_budget    
    for asset in self.assets.values():
        # Cuantas acciones son un paquete?
        if asset.price >= min_money_package:
            base_units = 1
        else:
            base_units = round(min_money_package / asset.price)

        # Cuantos paquetes nos entran en el presupuesto?
        max_packages =  math.floor(max_amt / (asset.price*base_units))

        # Si no entra ni siquiera un paquete, no añadimos esta variable
        if max_packages < 1:
            continue

        # Tenemos que sumar 1 ya que si el número máximo es justo una potencia de dos
        # no cubriríamos todo (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
        k = int(math.floor((math.log2(max_packages))+1))

        for i in range(k):
            package_multiplier = 2**i
            real_units = package_multiplier * base_units
            cost = real_units * asset.price
            val = cost * asset.roi
            self.variables.append(DecisionVariable(asset.name, real_units, cost, val))

In [ ]:
@add_method(PortfolioProblem)
def create_variables_with_bounds(self, bounds, min_package_pct=0.05):
    """
    Genera variables para cubrir un rango específico (coste de las acciones) [min, max] por activo.
    
    Para cada activo crea:
    - Una variable 'base' que cubre el mínimo solicitado.
    - Variables binarias (paquetes) para cubrir la diferencia (max - min).

    El parámetro bounds es un diccionario. Las claves son los nombres de los activos
    y los valores tuplas que contienen el mínimo y el máximo: name -> (min, max)
    """
    min_money_package = self.max_budget * min_package_pct

    for name, asset in self.assets.items():
        if name not in bounds:
            continue #Si no está en la aproximación, no permitimos añadirla a la solución
        else:
            min_amt, max_amt = bounds[name]
        
        # Limitamos el máximo
        if max_amt > self.max_budget:
            max_amt = self.max_budget

        # Para la parte binaria, qué incrementos usamos?
        if asset.price >= min_money_package:
            base_units = 1
        else:
            base_units = round(min_money_package / asset.price)
        print ("Base units: ", base_units)

        # Creamos la primera variable con el valor mínimo
        # Calculamos cuántas acciones caben en el importe mínimo
        units_at_min = math.floor(min_amt / asset.price)

        print (" units_at_min: ", units_at_min)
        if units_at_min > 0:
            cost = units_at_min * asset.price
            val = cost * asset.roi
            print ("cost", cost, "val", val)
            # Esta variable representa el bloque "fijo" o mínimo esperado
            self.variables.append(DecisionVariable(asset.name, 1.0, cost, val))

        # Creamos las variables binarias hasta llegar al máximo
        # Calculamos el espacio que queda entre el máximo y lo que ya cubrimos con el mínimo
        units_at_max = math.floor(max_amt / asset.price)
        delta_units = units_at_max - units_at_min

        # Convertimos ese delta a número de "paquetes"
        packages_in_delta = delta_units // base_units

        if packages_in_delta < 1:
            # Si la diferencia es menor que un paquete base, no añadimos variables extra
            continue

        # Tenemos que sumar 1 ya que si el número máximo es justo una potencia de dos
        # no cubriríamos todo (log2(8)=3, 2^0 + 2^1 +2^2 = 7)
        k = int(math.floor(math.log2(packages_in_delta)+1))
        
        for i in range(k):
            package_multiplier = 2**i
            real_units = package_multiplier * base_units
            cost = real_units * asset.price
            val = cost * asset.roi
            self.variables.append(DecisionVariable(asset.name, real_units, cost, val))